In [ ]:
from pandas import DataFrame
from pandas import read_csv
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import StandardScaler

/home/sudaraka/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
dataset = read_csv('with_timestamp.csv', header=0, index_col=0)
dataset.drop("@timestamp", axis=1, inplace=True)
dataset.head()

## Normalizing Data

In [ ]:
dataset[dataset.columns] = StandardScaler().fit_transform(dataset)

## Shifting the dataset
shifting only one minute

In [ ]:
dataset_shifted = dataset.shift(-6)
dataset = dataset.iloc[:-6]
dataset_shifted = dataset_shifted.iloc[:-6]

## Spliting the dataset 
Ratio - 80 : 20

In [ ]:
train = dataset.shape[0] * 2 // 10
    
train_dataset = dataset.iloc[:-train]
train_dataset_shifted = dataset_shifted.iloc[:-train]

test_dataset = dataset.iloc[-train:]
test_dataset_shifted = dataset_shifted.iloc[-train:]

In [ ]:
train_dataset = train_dataset.values.reshape((train_dataset.shape[0], 1, train_dataset.shape[1]))
print(train_dataset_shifted.shape)

## Training Models

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(10, input_shape=(1, train_dataset_shifted.shape[1])))
model_lstm.add(Dense(44))
model_lstm.compile(loss='mae', optimizer='adam')

model_lstm.fit(train_dataset, train_dataset_shifted, verbose=2, shuffle=False)

## Predicting 

In [ ]:
test_dataset_predicted = model_lstm.predict(test_dataset.values.reshape((test_dataset.shape[0], 1, test_dataset.shape[1])))

In [ ]:
test_dataset_predicted_pd = DataFrame(test_dataset_predicted)
test_dataset_predicted_pd = test_dataset_predicted_pd.set_index(test_dataset_shifted.index)

In [ ]:
f = pyplot.figure()
for i in range(test_dataset_shifted.shape[1]):
    pyplot.plot(test_dataset_shifted[test_dataset_shifted.columns[i]], label='real')
    pyplot.plot(test_dataset_predicted_pd[test_dataset_predicted_pd.columns[i]], label='lstm')
    pyplot.legend()
    pyplot.title(test_dataset_shifted.columns[i])
    pyplot.show()
    print("Mean Absolute Error : ",mean_absolute_error(test_dataset_shifted[test_dataset_shifted.columns[i]], test_dataset_predicted_pd[test_dataset_predicted_pd.columns[i]]))